In [5]:
import torch
from transformers.modeling_camembert import CamembertForMaskedLM
from transformers.tokenization_camembert import CamembertTokenizer
import numpy as np

In [2]:
# CamemBERT-base model & tokenizer
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")

Some weights of CamembertForMaskedLM were not initialized from the model checkpoint at camembert-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def ComputeContextualSurprise(list_fname= None,output_name = None, model = None,mask=None, tokenizer= None, topk=None):
    sentence_file = open(list_fname,'r', encoding='utf-8')
    output_fname = "/home/jeremy/Desktop/"+ output_name+".csv"
    output_file = open(output_fname,'w')
    sentences = sentence_file.readlines()
    for s in sentences:
        split_input = s.split(" ")
        Target_outputs = []
        sentence_score = 0;
        nbwords = 0;
        for i,w in enumerate(s.split(" ")):
            target_token = w
            nbwords = nbwords + 1
            masked_input = split_input.copy()
            masked_input[i] = mask
            masked_input = " ".join(masked_input)
            assert masked_input.count(mask) == 1
            input_ids = torch.tensor(tokenizer.encode(masked_input, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
            logits = model(input_ids)[0] #[:, 0, :] # The last hidden-state is the first element of the output tuple
            masked_index = (input_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()
            logits = logits[0, masked_index, :]
            prob = logits.softmax(dim=0)
            values, indices = prob.topk(k=topk, dim=0)
            topk_predicted_token_bpe = " ".join(
                [tokenizer.convert_ids_to_tokens(indices[i].item()) for i in range(len(indices))]
            )
            masked_token = tokenizer.mask_token
            for index, predicted_token_bpe in enumerate(topk_predicted_token_bpe.split(" ")):
                predicted_token = predicted_token_bpe.replace("\u2581", " ")
                if (predicted_token.casefold().strip() == target_token.casefold().strip()):
                    sentence_score = sentence_score +  (-np.log2(values[index].item()))
                    Target_outputs.append(
                        (
                            values[index].item(),
                            predicted_token,
                        )
                    )
        output_file.write(str(sentence_score/nbwords)+"\t"+ str(sentence_score) + "\n")
    sentence_file.close()
    output_file.close()
    print(output_name + ' Done !')
    return 0              

In [16]:
ComputeContextualSurprise(list_fname="/home/jeremy/Downloads/LM/list.txt",output_name = "CamemBERTfinal",model = model,mask='<mask>', tokenizer= tokenizer, topk=3000)

CamemBERTfinal Done !


0

In [ ]:
# Adapted from https://camembert-model.fr/